In [6]:
import numpy as np
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
import torch

from sklearn.metrics import f1_score

In [8]:
files = ['train', 'valid', 'test']
datasets = []
for file in files:
    with open(f'../data/preprocessed/{file}_transformers.npy', 'rb') as f:
        datasets.append(np.load(f))

train_corpus, valid_corpus, test_corpus = datasets
print(f'Train set shape: {train_corpus.shape}\nValidation set shape: {valid_corpus.shape}\nTest set shape: {test_corpus.shape}')

Train set shape: (9682, 252)
Validation set shape: (3227, 252)
Test set shape: (3228, 252)


In [10]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')